In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from collections import Counter
from sklearn import preprocessing
import scipy

In [4]:
ls data

crime_astros_finalA.csv*  crime_rockets_final.csv*  crime_texans_final.csv*
crime_astros_final.csv*   crime_ru_final.csv*       crime_uh_finalA.csv*
crime_dynamo_finalA.csv*  crime_ru_final.csvA.csv*  crime_uh_final.csv*
crime_dynamo_final.csv*   crime_texans_finalA.csv*


In [5]:
path = 'data/crime_rockets_final.csv'

In [6]:
df = pd.read_csv(path)
df=df.rename(columns = {'Unnamed: 0':'date','Hour':'hour'})
df.hour = df.hour.astype(int)
df.day = df.day.astype(int)
df.year = df.year.astype(int)
df.date = pd.to_datetime(df.date)  # datetime index
df = df.set_index('date').sort_index(ascending=True)
#df['game_on'] = np.where(df.game =='No Game',0,1)
#df.drop(['game'],axis=1,inplace=True)
df.head(2)

,Beat,OffenseType,Premise,NumOffenses,hour,day,weekday,month,year,lat,lon,add,dist_stadium,game
date,,,,,,,,,,,,,,
2010-01-01,1A10,Auto Theft,bar/night club parking lot,1,0,1,Friday,Jan,2010,29.757212,-95.353294,"350 Hamilton St, Houston, TX 77002, USA",0.693506,No Game
2010-01-01,1A10,Theft,bar/night club,1,0,1,Friday,Jan,2010,29.753512,-95.363811,"1250 Caroline St, Houston, TX 77002, USA",0.207741,No Game


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39513 entries, 2010-01-01 to 2017-12-31
Data columns (total 14 columns):
Beat            39513 non-null object
OffenseType     39513 non-null object
Premise         39513 non-null object
NumOffenses     39513 non-null int64
hour            39513 non-null int64
day             39513 non-null int64
weekday         39513 non-null object
month           39513 non-null object
year            39513 non-null int64
lat             39513 non-null float64
lon             39513 non-null float64
add             39513 non-null object
dist_stadium    39513 non-null float64
game            39513 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 4.5+ MB


In [8]:
df = df[[ 'OffenseType', 'Premise', 'hour', 'weekday', 'month', 'year',  'dist_stadium', 'game']]

In [9]:
df.game.value_counts()

No Game      35025
Won Game      2967
Lost Game     1521
Name: game, dtype: int64

## Cleanup premise column

In [10]:
len(df.Premise.value_counts())

186

In [11]:
df.Premise.value_counts()[:10]

road/street/sidewalk                10171
residence/house                      3020
other parking lot                    2261
no label                             1771
commercial parking lot/garage        1759
apartment                            1399
commercial parking lot or garage     1136
other/unknown                        1127
apartment parking lot                1005
bar/night club                        749
Name: Premise, dtype: int64

In [12]:
# remove slash, commas, parenthesis and empty spaces
df.Premise = df.Premise.str.replace('/', ' ').str.replace(',', ' ').str.replace('(', '').str.replace(')', '').str.strip()

In [13]:
len(df.Premise.value_counts())

185

In [14]:
df.Premise.value_counts()

road street sidewalk                                     10171
residence house                                           3020
other parking lot                                         2261
no label                                                  1771
commercial parking lot garage                             1759
apartment                                                 1399
commercial parking lot or garage                          1136
other unknown                                             1127
apartment parking lot                                     1005
bar night club                                             749
grocery supermarket                                        711
department discount store                                  687
restaurant cafeteria parking lot                           686
driveway                                                   599
office building                                            525
bar or night club                                      

In [15]:
Counter(" ".join(df.Premise).split()).most_common(50)

[('road', 10198),
 ('street', 10171),
 ('sidewalk', 10171),
 ('parking', 8570),
 ('lot', 8570),
 ('or', 4017),
 ('other', 3860),
 ('commercial', 3456),
 ('residence', 3156),
 ('garage', 3143),
 ('house', 3112),
 ('apartment', 2476),
 ('store', 2343),
 ('club', 2114),
 ('bar', 1946),
 ('night', 1946),
 ('no', 1771),
 ('label', 1771),
 ('unknown', 1541),
 ('restaurant', 1392),
 ('cafeteria', 1377),
 ('building', 1223),
 ('grocery', 1215),
 ('supermarket', 1215),
 ('hotel', 864),
 ('motel', 864),
 ('etc.', 824),
 ('station', 819),
 ('bus', 697),
 ('convenience', 694),
 ('department', 687),
 ('discount', 687),
 ('office', 610),
 ('driveway', 599),
 ('business', 590),
 ('non-specific', 570),
 ('pool', 515),
 ('vacant', 437),
 ('not', 423),
 ('service', 420),
 ('listed', 417),
 ('gas', 401),
 ('parks', 351),
 ('recreation', 351),
 ('zoo', 351),
 ('center', 347),
 ('stop', 315),
 ('inn', 305),
 ('multi-plex', 300),
 ('triplex', 300)]

# cleanup

In [16]:
com = df.Premise.str.contains('church')
df[com].Premise.value_counts()

church synagogue temple                     134
church  synagogue  or temple                 50
church  synagogue  or temple parking lot     49
church synagogue temple parking lot          42
vacant church synagogue temple                5
vacant church  synagogue  or temple           5
Name: Premise, dtype: int64

In [17]:

df.Premise = df.Premise.str.replace('church synagogue temple', 'church')
df.Premise = df.Premise.str.replace('church  synagogue  or temple', 'church')
df.Premise = df.Premise.str.replace('vacant church synagogue temple', 'church')
df.Premise = df.Premise.str.replace('vacant church  synagogue  or temple', 'church')
df.Premise = df.Premise.str.replace('vacant church', 'church')
df.Premise = df.Premise.str.replace('church parking lot', 'church prk')

com = df.Premise.str.contains('church')
df[com].Premise.value_counts()

church        194
church prk     91
Name: Premise, dtype: int64

In [18]:
com = df.Premise.str.contains('street')
df[com].Premise.value_counts()

road street sidewalk    10171
Name: Premise, dtype: int64

In [19]:
df.Premise = df.Premise.str.replace('road street sidewalk', 'street')


In [20]:
com = df.Premise.str.contains('commercial')
df[com].Premise.value_counts()

commercial parking lot garage       1759
commercial parking lot or garage    1136
commercial building                  465
vacant building commercial            80
commercial or training school         16
Name: Premise, dtype: int64

In [21]:
df.Premise = df.Premise.str.replace('commercial parking lot garage', 'comm prk')
df.Premise = df.Premise.str.replace('commercial parking lot or garage', 'comm prk')
df.Premise = df.Premise.str.replace('commercial building', 'comm bld')
df.Premise = df.Premise.str.replace('vacant building commercial', 'comm bld')
df.Premise = df.Premise.str.replace('commercial or training school', 'comm bld')

In [22]:
com = df.Premise.str.contains('comm')
df[com].Premise.value_counts()

comm prk            2895
comm bld             561
mall common area      29
Name: Premise, dtype: int64

In [23]:
## hotel/motel

hot = df.Premise.str.contains('hotel')
df[hot].Premise.value_counts()

df.Premise = df.Premise.str.replace('hotel motel etc.', 'hotel')
df.Premise = df.Premise.str.replace('hotel  motel  inn  etc.', 'hotel')
df.Premise = df.Premise.str.replace('hotel or motel parking lot', 'hotel prk')
df.Premise = df.Premise.str.replace('hotel motel parking lot', 'hotel prk')
df.Premise = df.Premise.str.replace('vacant hotel motel etc.', 'hotel')
df.Premise = df.Premise.str.replace('vacant hotel  motel  etc.', 'hotel')
df.Premise = df.Premise.str.replace('vacant hotel', 'hotel')


hot = df.Premise.str.contains('hotel')
df[hot].Premise.value_counts()

hotel        612
hotel prk    252
Name: Premise, dtype: int64

In [24]:
## apartments
wrd = df.Premise.str.contains('apartment')
df[wrd].Premise.value_counts()

apartment                                                      1399
apartment parking lot                                          1005
apartment rental office                                          31
vacant other residential apartment dorms                         18
vacant other residential apartment inn dorms boarding house      13
apartment laundry                                                10
Name: Premise, dtype: int64

In [25]:
df.Premise = df.Premise.str.replace('apartment parking lot', 'apartment prk')
df.Premise = df.Premise.str.replace('apartment rental office', 'apartment')
df.Premise = df.Premise.str.replace('vacant other residential apartment dorms', 'apartment')
df.Premise = df.Premise.str.replace('vacant other residential apartment inn dorms boarding house', 'apartment')
df.Premise = df.Premise.str.replace('apartment laundry', 'apartment')

wrd = df.Premise.str.contains('apartment')
df[wrd].Premise.value_counts()

apartment        1471
apartment prk    1005
Name: Premise, dtype: int64

In [26]:
## house
wrd = df.Premise.str.contains('house')
df[wrd].Premise.value_counts()

residence house                                                       3020
vacant single occupancy residence houses townhouses duplexes  etc.     136
vacant single occ resdhouse townhs dplex                                83
gym  recreat  club house  indoor pool  spa                              61
warehouse                                                               54
gym  recreat  club house  indoor pool                                   18
vacant storage fac barn garage warehouse                                 5
vacant storage facility barns garages warehouses etc.                    2
Name: Premise, dtype: int64

In [27]:
df.Premise = df.Premise.str.replace('residence house', 'house')
df.Premise = df.Premise.str.replace('vacant single occupancy residence houses townhouses duplexes  etc.', 'house')
df.Premise = df.Premise.str.replace('vacant single occ resdhouse townhs dplex', 'house')
df.Premise = df.Premise.str.replace('vacant single occupancy houses townhouses duplexes  etc.', 'house')

wrd = df.Premise.str.contains('house')
df[wrd].Premise.value_counts()

house                                                    3239
gym  recreat  club house  indoor pool  spa                 61
warehouse                                                  54
gym  recreat  club house  indoor pool                      18
vacant storage fac barn garage warehouse                    5
vacant storage facility barns garages warehouses etc.       2
Name: Premise, dtype: int64

In [28]:
# warehouse
wrd = df.Premise.str.contains('warehouse')
df[wrd].Premise.value_counts()


warehouse                                                54
vacant storage fac barn garage warehouse                  5
vacant storage facility barns garages warehouses etc.     2
Name: Premise, dtype: int64

In [29]:
df.Premise = df.Premise.str.replace('vacant storage fac barn garage warehouse', 'warehouse')


wrd = df.Premise.str.contains('warehouse')
df[wrd].Premise.value_counts()

warehouse                                                59
vacant storage facility barns garages warehouses etc.     2
Name: Premise, dtype: int64

In [30]:
# bar
wrd = df.Premise.str.contains('bar')
df[wrd].Premise.value_counts()

bar night club                                           749
bar or night club                                        515
bar night club parking lot                               400
bar or night club parking lot                            282
barber and beauty shops                                   24
vacant storage facility barns garages warehouses etc.      2
Name: Premise, dtype: int64

In [31]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('bar night club', 'bar_nc')

df.Premise = df.Premise.str.replace('bar or night club', 'bar_nc')

df.Premise = df.Premise.str.replace('bar or night club parking lot', 'bar_nc prk')

wrd = df.Premise.str.contains('bar')
df[wrd].Premise.value_counts()

bar_nc                                                   1264
bar_nc parking lot                                        682
barber and beauty shops                                    24
vacant storage facility barns garages warehouses etc.       2
Name: Premise, dtype: int64

In [32]:
# parking

wrd = df.Premise.str.contains('parking')
df[wrd].Premise.value_counts()

other parking lot                           2261
restaurant cafeteria parking lot             686
bar_nc parking lot                           682
convenience store parking lot                258
strip business center parking lot            139
grocery store or supermarket parking lot      80
grocery supermarket parking lot               67
hospital parking lot                          65
bank saving institution parking lot           29
mall parking lot                              19
liquor store parking lot                      15
bank or savings institution parking lot       13
sexually oriented business parking lot        10
laundry or dry cleaners parking lot            2
laundry dry cleaners parking lot               1
Name: Premise, dtype: int64

In [33]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('other parking lot', 'prk')
df.Premise = df.Premise.str.replace('restaurant cafeteria parking lot', 'rest prk')
df.Premise = df.Premise.str.replace('convenience store parking lot', 'store prk')
df.Premise = df.Premise.str.replace('strip business center parking lot', 'store prk')
df.Premise = df.Premise.str.replace('hospital parking lot', 'prk')
df.Premise = df.Premise.str.replace('grocery store or supermarket parking lot', 'store prk')
df.Premise = df.Premise.str.replace('grocery supermarket parking lot', 'store prk')
df.Premise = df.Premise.str.replace('church synagogue temple parking lot', 'church prk')
df.Premise = df.Premise.str.replace('bank saving institution parking lot', 'bank prk')
df.Premise = df.Premise.str.replace('bank or savings institution parking lot', 'bank prk')
df.Premise = df.Premise.str.replace('mall parking lot', 'store prk')
df.Premise = df.Premise.str.replace('sexually oriented business parking lot', 'store prk')
df.Premise = df.Premise.str.replace('church  synagogue  or temple parking lot', 'church prk')
df.Premise = df.Premise.str.replace('laundry or dry cleaners parking lot', 'store prk')
df.Premise = df.Premise.str.replace('liquor store parking lot', 'store prk')
df.Premise = df.Premise.str.replace('laundry dry cleaners parking lot', 'store prk')
df.Premise = df.Premise.str.replace('bar_nc parking lot', 'bar_nc prk')

wrd = df.Premise.str.contains('parking')
df[wrd].Premise.value_counts()

Series([], Name: Premise, dtype: int64)

In [34]:
# building
wrd = df.Premise.str.contains('building')
df[wrd].Premise.value_counts()

office building                                                                     525
government public building                                                           76
government or public building                                                        60
vacant other structure out buildings monuments buildings under construction etc.     13
vacant government or public building                                                  5
maintenance building services                                                         5
maintenance or building services                                                      4
vacant government public building                                                     3
Name: Premise, dtype: int64

In [35]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('office building', 'office bld')
df.Premise = df.Premise.str.replace('government public building', 'public bld')
df.Premise = df.Premise.str.replace('government or public building', 'public bld')
df.Premise = df.Premise.str.replace('maintenance building services', 'office bld')
df.Premise = df.Premise.str.replace('maintenance or building services', 'office bld')

df.Premise = df.Premise.str.replace('vacant government or public building', 'public bld')
df.Premise = df.Premise.str.replace('vacant other structure out buildings monuments buildings under construction etc.', 'office bld')

wrd = df.Premise.str.contains('building')
df[wrd].Premise.value_counts()

Series([], Name: Premise, dtype: int64)

In [36]:
# store

wrd = df.Premise.str.contains('store')
df[wrd].Premise.value_counts()

department discount store                    687
store prk                                    591
convenience store                            436
grocery store or supermarket                 348
specialty store non-specific                 129
drug store medical supply                    126
clothing store                               101
vehicle auto sales lease auto parts store     72
drug store or medical supply                  32
liquor store                                  26
electronics store  electrical sup.            23
vacant grocery store or supermarket            7
jewelry stores                                 5
electronics store  electrical supplies         2
furniture  appliances  radios  tv store        1
toys  arts craft musical  bike pet stores      1
Name: Premise, dtype: int64

In [37]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('department discount store', 'store')
df.Premise = df.Premise.str.replace('convenience store', 'store')
df.Premise = df.Premise.str.replace('clothing store', 'store')
df.Premise = df.Premise.str.replace('specialty store non-specific', 'store')
df.Premise = df.Premise.str.replace('vehicle auto sales lease auto parts store', 'store')
df.Premise = df.Premise.str.replace('drug store medical supply', 'store')
df.Premise = df.Premise.str.replace('liquor store', 'store')
df.Premise = df.Premise.str.replace('electronics store  electrical supply.', 'store')
df.Premise = df.Premise.str.replace('jewelry stores', 'store')
df.Premise = df.Premise.str.replace('electronics store  electrical supplies', 'store')
df.Premise = df.Premise.str.replace('vacant grocery store or supermarket', 'store')
df.Premise = df.Premise.str.replace('toys  arts craft musical  bike pet stores', 'store')
df.Premise = df.Premise.str.replace('grocery store or supermarket', 'store')
df.Premise = df.Premise.str.replace('drug store or medical supply', 'store')
#df.Premise = df.Premise.str.replace('storelies', 'store')

wrd = df.Premise.str.contains('store')
df[wrd].Premise.value_counts()

store                                      1972
store prk                                   591
electronics store  electrical sup.           23
furniture  appliances  radios  tv store       1
Name: Premise, dtype: int64

In [38]:
# restaurant
wrd = df.Premise.str.contains('rest')
df[wrd].Premise.value_counts()

rest prk                      686
restaurant cafeteria          414
restaurant or cafeteria       277
field  woods  forest  park     44
vacant restaurant              15
Name: Premise, dtype: int64

In [39]:
df.Premise = df.Premise.str.replace('restaurant cafeteria', 'rest')
df.Premise = df.Premise.str.replace('restaurant or cafeteria', 'rest')
df.Premise = df.Premise.str.replace('vacant restaurant', 'rest')

wrd = df.Premise.str.contains('rest')
df[wrd].Premise.value_counts()

rest                          706
rest prk                      686
field  woods  forest  park     44
Name: Premise, dtype: int64

In [40]:
#unk

wrd = df.Premise.str.contains('unknown')
df[wrd].Premise.value_counts()

other unknown                    1127
other  unknown  or not listed     414
Name: Premise, dtype: int64

In [41]:



#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('other unknown', 'unk')
df.Premise = df.Premise.str.replace('other  unknown  or not listed', 'unk')
df.Premise = df.Premise.str.replace('---', 'unk')

wrd = df.Premise.str.contains('unknown')
df[wrd].Premise.value_counts()

Series([], Name: Premise, dtype: int64)

In [42]:
# miscellaneous


wrd = df.Premise.str.contains('misc')
df[wrd].Premise.value_counts()

miscellaneous business non-specific    268
misc. business non-specific            173
Name: Premise, dtype: int64

In [43]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('miscellaneous business non-specific', 'store')
df.Premise = df.Premise.str.replace('misc. business non-specific', 'store')



wrd = df.Premise.str.contains('misc')
df[wrd].Premise.value_counts()

Series([], Name: Premise, dtype: int64)

In [44]:
## stadium

wrd = df.Premise.str.contains('stadium')
df[wrd].Premise.value_counts()

stadium  sports arena  race track    135
stadium sprts arena race track       105
Name: Premise, dtype: int64

In [45]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('stadium  sports arena  race track', 'stadium')
df.Premise = df.Premise.str.replace('stadium sprts arena race track', 'stadium')


wrd = df.Premise.str.contains('stadium')
df[wrd].Premise.value_counts()

stadium    240
Name: Premise, dtype: int64

In [46]:
# no label

wrd = df.Premise.str.contains('no')
df[wrd].Premise.value_counts()

no label                    1771
train not metro terminal       6
not listed                     3
Name: Premise, dtype: int64

In [47]:

#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('no label', 'unk')
df.Premise = df.Premise.str.replace('not listed', 'unk')


wrd = df.Premise.str.contains('no')
df[wrd].Premise.value_counts()

train not metro terminal    6
Name: Premise, dtype: int64

In [48]:
# park

wrd = df.Premise.str.contains('park')
df[wrd].Premise.value_counts()


parks & recreation  zoo  swim pool           214
parks and recreation  zoo  swimming pool     137
field  woods  forest  park                    44
park and ride terminal                        20
amusement park  bowling alley  skate rink     15
amuse. park bowl. alley skate rink            14
park & ride terminal                           7
Name: Premise, dtype: int64

In [49]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('parks & recreation  zoo  swim pool', 'parks')
df.Premise = df.Premise.str.replace('parks and recreation  zoo  swimming pool', 'parks')
df.Premise = df.Premise.str.replace('field  woods  forest  park', 'parks')

df.Premise = df.Premise.str.replace('amuse. park bowl. alley skate rink', 'parks')
df.Premise = df.Premise.str.replace('amusement park  bowling alley  skate rink', 'parks')



wrd = df.Premise.str.contains('park')
df[wrd].Premise.value_counts()

parks                     424
park and ride terminal     20
park & ride terminal        7
Name: Premise, dtype: int64

In [50]:
# garage

wrd = df.Premise.str.contains('garage')
df[wrd].Premise.value_counts()

garage or carport                                        151
garage carport                                            92
vacant storage facility barns garages warehouses etc.      2
Name: Premise, dtype: int64

In [51]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('garage or carport', 'prk')
df.Premise = df.Premise.str.replace('garage carport', 'prk')


wrd = df.Premise.str.contains('garage')
df[wrd].Premise.value_counts()

vacant storage facility barns garages warehouses etc.    2
Name: Premise, dtype: int64

In [52]:
# bus /metro

wrd = df.Premise.str.contains('rail')
df[wrd].Premise.value_counts()

light rail platform               90
light rail metro rail vehicle     76
railroad track right of way        8
railroad track or right of way     5
light rail vehicle                 4
Name: Premise, dtype: int64

In [53]:
wrd = df.Premise.str.contains('university')
df[wrd].Premise.value_counts()

university college                     92
university or college                  38
vacant school or college university    11
Name: Premise, dtype: int64

In [54]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('university college', 'college')
df.Premise = df.Premise.str.replace('university or college', 'college')
df.Premise = df.Premise.str.replace('vacant school or college university', 'college')
wrd = df.Premise.str.contains('college')
df[wrd].Premise.value_counts()

college                  141
vacant school college      2
Name: Premise, dtype: int64

In [55]:
wrd = df.Premise.str.contains('school')
df[wrd].Premise.value_counts()

high school              112
elementary school         15
private school            13
middle school             10
vacant school college      2
Name: Premise, dtype: int64

In [56]:

df.Premise = df.Premise.str.replace('high school', 'school')
df.Premise = df.Premise.str.replace('private school', 'school')
df.Premise = df.Premise.str.replace('elementary school', 'school')
df.Premise = df.Premise.str.replace('middle school', 'school')

wrd = df.Premise.str.contains('school')
df[wrd].Premise.value_counts()


school                   150
vacant school college      2
Name: Premise, dtype: int64

In [57]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('bus stop', 'bus_rail')
df.Premise = df.Premise.str.replace('bus station', 'bus_rail')
df.Premise = df.Premise.str.replace('light rail metro rail vehicle', 'bus_rail')
df.Premise = df.Premise.str.replace('train not metro terminal', 'bus_rail')
df.Premise = df.Premise.str.replace('light rail platform', 'bus_rail')
df.Premise = df.Premise.str.replace('light rail metro rail vehicle', 'bus_rail')
df.Premise = df.Premise.str.replace('railroad track right of way', 'bus_rail')
df.Premise = df.Premise.str.replace('light rail vehicle', 'bus_rail')
df.Premise = df.Premise.str.replace('railroad track or right of way', 'bus_rail')
df.Premise = df.Premise.str.replace('park and ride terminal', 'bus_rail')
df.Premise = df.Premise.str.replace('park & ride terminal', 'bus_rail')


wrd = df.Premise.str.contains('rail')
df[wrd].Premise.value_counts()

bus_rail    913
Name: Premise, dtype: int64

In [58]:
## convention

wrd = df.Premise.str.contains('conv')
df[wrd].Premise.value_counts()

convention center or exhibit halls    98
convention center exhibit halls       88
Name: Premise, dtype: int64

In [59]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('convention center or exhibit halls', 'convention')
df.Premise = df.Premise.str.replace('convention center exhibit halls', 'convention')


wrd = df.Premise.str.contains('conv')
df[wrd].Premise.value_counts()

convention    186
Name: Premise, dtype: int64

In [60]:
df.Premise = df.Premise.str.replace('libraries  museums', 'libraries')


In [61]:
#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('service gas station', 'store')
df.Premise = df.Premise.str.replace('service or gas station', 'store')
df.Premise = df.Premise.str.replace('utility company electric gas water', 'public bld')
df.Premise = df.Premise.str.replace('utility company  electric  gas  water', 'public bld')


#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('condominium', 'house')



#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('multi-plex home duplex triplex etc.', 'house')
df.Premise = df.Premise.str.replace('mobile home', 'house')
df.Premise = df.Premise.str.replace('nursing home', 'house')
df.Premise = df.Premise.str.replace('multi-plex hmeduplex triplex etc', 'house')


#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('gym recreat club hse indr pool spa', 'gym')
df.Premise = df.Premise.str.replace('gym  recreat  club house  indoor pool  spa', 'gym')
df.Premise = df.Premise.str.replace('gym  recreat  club house  indoor pool', 'gym')

#df.Premise = df.Premise.str.replace('', '')
df.Premise = df.Premise.str.replace('grocery supermarket', 'store')
df.Premise = df.Premise.str.replace('pawn  resale shop  or flea market', 'store')
df.Premise = df.Premise.str.replace('pawn resale shop flea market', 'store')

df.Premise = df.Premise.str.replace('construction site', 'constr')


In [62]:
len(df.Premise.value_counts())

95

In [63]:
path 

'data/crime_rockets_final.csv'

In [64]:
df.to_csv('data/crime_rockets_finalA.csv')

In [65]:
df.Premise.value_counts()

street                                                   10171
house                                                     3707
store                                                     3519
unk                                                       3432
comm prk                                                  2895
prk                                                       2569
apartment                                                 1471
bar_nc                                                    1264
apartment prk                                             1005
bus_rail                                                   913
rest                                                       706
rest prk                                                   686
bar_nc prk                                                 682
hotel                                                      612
driveway                                                   599
store prk                                              